In [1]:
import requests
import pandas as pd
from io import StringIO
import json
from matplotlib import pyplot
import unicodedata
%matplotlib inline

In [2]:
def get_insee(commune):
    """
    returns the insee code of the commune
    """
    commune_ = unicodedata.normalize('NFD', commune.lower()).encode('ascii', 'ignore').decode("utf-8")
    url = 'https://geo.api.gouv.fr/communes?nom={c}&fields=nom,code,codesPostaux,'\
              'codeDepartement,codeRegion,population&format=json&geometry=centre'\
              .format(c=commune_)
    exists = len(json.loads(requests.get(url).text))
    if exists>0 :
        return json.loads(requests.get(url).text)[0]["code"]
    else:
        print("Commune does not exist")
        return -1

In [3]:
def get_bss(code_commune):
    url =  "https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/stations?code_commune={c}&format=json&size=20".format(c=code_commune)
    exists = json.loads(requests.get(url).text)["count"]
    if exists>0 :
        data = json.loads(requests.get(url).text)
        bss = [station["code_bss"] for station in data["data"]]
        return bss

    else:
        print("Code commune does not exist")
        return -1

In [4]:
def get_station_piezo(station, start_date=None, end_date=None):
    url = "https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={bss}&date_debut_mesure={d1}&date_fin_mesure={d2}&size=1".format(bss=station, d1=start_date, d2=end_date)
    number = json.loads(requests.get(url).text)["count"]
    if number > 0:
        url = "https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={bss}&date_debut_mesure={d1}&date_fin_mesure={d2}&size={s}".format(bss=station, d1=start_date, d2=end_date, s=number + 1)
        return json.loads(requests.get(url).text)

In [5]:
c = get_insee("Orléans")
c

'45234'

In [6]:
bss  = get_bss(c)
bss

['03635X0545/PZ1',
 '03636X1060/PZ2',
 '03636X1061/PZ3',
 '03636X1062/PZ4',
 '03982X1045/F']

In [7]:
mesure = get_station_piezo(bss[0], start_date = "2019-01-12", end_date = "2019-01-30")
mesure

{'count': 18,
 'first': 'https://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=03635X0545/PZ1&date_debut_mesure=2019-01-12&date_fin_mesure=2019-01-30&page=1&size=19',
 'last': None,
 'prev': None,
 'next': None,
 'api_version': '1.4.0',
 'data': [{'code_bss': '03635X0545/PZ1',
   'urn_bss': 'http://services.ades.eaufrance.fr/pointeau/03635X0545/PZ1',
   'date_mesure': '2019-01-12',
   'timestamp_mesure': 1547334000000,
   'niveau_nappe_eau': 88.39,
   'mode_obtention': 'Valeur mesurée',
   'statut': 'Donnée contrôlée niveau 2',
   'qualification': 'Correcte',
   'code_continuite': '2',
   'nom_continuite': 'Point lié au point précédent',
   'code_producteur': '293',
   'nom_producteur': 'Service Géologique Régional Centre (293)',
   'code_nature_mesure': None,
   'nom_nature_mesure': None,
   'profondeur_nappe': 4.95},
  {'code_bss': '03635X0545/PZ1',
   'urn_bss': 'http://services.ades.eaufrance.fr/pointeau/03635X0545/PZ1',
   'date_mesure': '2019-01-13',
   'timestamp

In [26]:
import predict_flair
from flair.models import SequenceTagger
import fr_core_news_lg as fr

def get_locations(query):
    
    batch_size = 4
    MODEL_PATH = "/home/maya/Documents/hubeau/inria/Code/NER_tool/stacked-standard-flair-150-wikiner.pt"
    tag_type = "label"
    model = SequenceTagger.load(MODEL_PATH)
    
    snippets = [[1, query]]
    result = predict_flair.get_entities(snippets, model, tag_type, batch_size)["snippets"][0][1]
    locations = [entity["text"] for entity in result["entities"] if entity["type"] == "LOC"]
    return locations

In [27]:
locations = get_locations("Quel est le niveau de la nappe phréatique à Orléans aujourd'hui?")
locations

2021-06-19 17:08:51,703 loading file /home/maya/Documents/hubeau/inria/Code/NER_tool/stacked-standard-flair-150-wikiner.pt


['Orléans']

In [11]:
locations = get_locations("A quelle profondeur se trouve la nappe à l'adresse 12 rue de Coulmiers, 45000 Orléans")
locations

2021-06-19 16:02:38,532 loading file /home/maya/Documents/hubeau/inria/Code/NER_tool/stacked-standard-flair-150-wikiner.pt
à quel profondeur se trouve le nappe à le adresse 12 rue de Coulmiers , 45000 Orléans


['Coulmiers', 'Orléans']

In [12]:
locations = get_locations("Y'a-t-il de l'eau dans le sous-sol parisien")
locations

2021-06-19 16:02:44,217 loading file /home/maya/Documents/hubeau/inria/Code/NER_tool/stacked-standard-flair-150-wikiner.pt
Y' avoir -t il de le eau dans le sous-sol parisien


[]

In [28]:
locations = get_locations("J'habite à Loury dans le Loiret, à quelle profondeur faut-il que je creuse un puits dans mon jardin pour avoir de l'eau ")
locations

2021-06-19 17:10:56,099 loading file /home/maya/Documents/hubeau/inria/Code/NER_tool/stacked-standard-flair-150-wikiner.pt


['Loury', 'Loiret']

In [43]:
from nltk.corpus import wordnet as wn
x = [synset.name() for synset in wn.synsets('Parisian')]
x

['parisian.n.01', 'parisian.a.01']

In [35]:
import nltk
nltk.download('omw')

[nltk_data] Downloading package omw to /home/maya/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.


True

In [7]:
from python_heideltime import Heideltime

heideltime_parser = Heideltime()
heideltime_parser.set_document_type('NEWS')
heideltime_parser.set_language('FRENCH')
print(heideltime_parser.parse("Quel est le niveau de la nappe phréatique à Orléans hier et aujourd'hui?"))

<?xml version="1.0"?>
<!DOCTYPE TimeML SYSTEM "TimeML.dtd">
<TimeML>
Quel est le niveau de la nappe phréatique à Orléans <TIMEX3 tid="t1" type="DATE" value="2021-06-17">hier</TIMEX3> et <TIMEX3 tid="t2" type="DATE" value="2021-06-18">aujourd'hui</TIMEX3>?
</TimeML>


